In [ ]:
from dotenv import get_key
from pymongo import MongoClient
import pymongo
import psycopg2 # Remover
from pprint import pprint  # Imprimir bsons formatados
from utils import *

STRING = get_key('.env', 'MG_STRING')
NAME = get_key('.env', 'PG_NAME')
USER = get_key('.env', 'PG_USER')
PASSWORD = get_key('.env', 'PG_PASSWORD')
HOST = get_key('.env', 'PG_HOST')
PORT = get_key('.env', 'PG_PORT')

## Funções para acesso ao MongoDB

In [ ]:
def create_mongo_connection():
	"""Cria uma conexão com o banco de dados MongoDB. Retorna o cliente."""
	try:
		uri = STRING
		client = MongoClient(uri, server_api=pymongo.server_api.ServerApi(version="1"))
		client.admin.command("ping")
		print("---- Conexão bem sucedida ----")
	except Exception as e:
		raise Exception(
			"The following error occurred: ", e)
	return client

def end_mongo_connection(client):
	client.close()
	print("---- Conexão encerrada -----")

## Criação de tabelas no banco

In [71]:
def create_tables(db, col):
	match col:
		case 'pais':
			db.command({
				"collMod": "pais",
				"validator": {
					"$jsonSchema": {
						"bsonType": "object",
						"properties": {
							"continente": {"bsonType": "string"},
							"codigo": {"bsonType": "string"},
							"nome": {"bsonType": "string"},
							"grupos": {
								"bsonType": "array",
								"items": {
									"bsonType": "object",
									"properties": {
										"sigla": {"bsonType": "string"}
									}
								}
							},
							"_id": {"bsonType": "objectId"}
						},
						"title": "pais"
					}
				},
				'validationLevel': 'moderate',
				'validationAction': 'warn'
			})
		case 'emissaoGas':
			collection_name = 'emissaoGas'
		case _:
			raise ValueError(f"Nome de coleção inválido: {col}")

In [ ]:
def consultas(client):
	# for db_info in client.list_database_names():
	# 	print(db_info)
	# db = client["Biblioteca"]
	# # collections = db.list_collection_names()
	# # print("Coleções em samples_mflix:", collections)
	# result = db.books.find({"authors": "Machado de Assis"}, {'_id': 0, 'title': 1})
	
	# for doc in result:
	# 	pprint(doc)

	db = client["projeto_mc536"]
	# colecao = db["teste"]
	# # db.teste.insertOne({'name': 'José',
	# # 				 'age': 58})
	# colecao.insert_one({'campo': 'valor exemplo'})
	# result = db['teste'].find({}, {'_id':0})
	# for doc in result:
	# 	pprint(doc)


	# result = db['teste'].find({}, {'_id':0})
	# print('---')
	# for doc in result:
	# 	pprint(doc)

	# Corrigindo a criação da coleção 'pais' caso ela não exista antes de aplicar o validator
	entries = {'pais': [
			{
				'continente': 18,
				'codigo': 'HW',
				'nome': 'Hogwarts',
				'grupos': [{'sigla': 'MAGIA'}],
				'preco' : 15
			}
		],
		'emissaoGas': [
			{
				'paisCodigo': 'BRA',
				'ano': 2005,
				'emissoes': [{
					'nome': 'Dióxido de Carbono',
					'formula': 'CO2',
					'valor': 320
				}]
			},
			{
				'paisCodigo': 'USA',
				'ano': 1998,
				'emissoes': [{
					'nome': 'Metano',
					'formula': 'CH4',
					'valor': 1000
				}]
			}
		]}
	
	for col in entries.keys():

		if col not in db.list_collection_names():
			db.create_collection(col)

		create_tables(db, col)
		db[col].delete_many({})
		db[col].insert_many(entries[col])

		print(f'> Coleção {col} <')
		# Exibindo os documentos inseridos
		for doc in db[col].find({}, {'_id': 0}):
			pprint(doc)
		
client = create_mongo_connection()
consultas(client)
end_mongo_connection(client)


---- Conexão bem sucedida ----
> Coleção pais <
{'codigo': 'HW',
 'continente': 18,
 'grupos': [{'sigla': 'MAGIA'}],
 'nome': 'Hogwarts',
 'preco': 15}
> Coleção emissaoGas <
{'ano': 2005,
 'emissoes': [{'formula': 'CO2', 'nome': 'Dióxido de Carbono', 'valor': 320}],
 'paisCodigo': 'BRA'}
{'ano': 1998,
 'emissoes': [{'formula': 'CH4', 'nome': 'Metano', 'valor': 1000}],
 'paisCodigo': 'USA'}
---- Conexão encerrada -----


In [ ]:
client = create_mongo_connection()
consultas(client)
end_mongo_connection(client)